In [1]:
import bw2io as bi
import bw2data as bd
import bw2calc as bc

## This notebook

The objective of this notebook is to demonstrate how to work with parameters and Monte Carlo on Activity-Browser.

- Author (and contact):
    - Alvaro Hahn Menacho (alvaro.hahn-menacho@psi.ch)
    - Romain Sacchi (romain.sacchi@psi.ch)


## Assigment

Congrats! You’ve just been hired as a life cycle assessment (LCA) expert at an innovative company that values both data transparency and open-source software. Your team is revisiting an old life cycle inventory (LCI) comparing the impacts of driving an electric vehicle (EV), a gasoline vehicle, and a hydrogen vehicle, particularly in terms of **Global Warming Potential (GWP)** and **particulate matter formation impacts on human health**.

However, this LCI has been considered unsuitable for decision-making due to its deterministic assumptions. Your role is to address these concerns by creating a more flexible, data-driven LCA model that allows for sensitivity analysis and deeper insight into key impact factors. During a team meeting, your colleagues raised the following points:

1. **Energy Consumption of Electric Vehicles:** One colleague pointed out that the stated consumption of around 20 kWh per 100 km for the EV might not be accurate across all cases. They emphasized the importance of reflecting varying data and assumptions in your model. Also, they stress the importance of test the sensitivity of results on different battery capacities and cell energy densities.

2. **Hydrogen fuel tank gravimetric density**: Another team member mentioned that the gravimetric density of hydrogen fuel tanks improved rapidly these past few years. Currently, the model assumes a gravimetric density index of 0.05, but they suggested that this value should be updated to reflect the latest advancements in hydrogen storage technology.

3. **Hydrogen blend**: You've seen some statistics recently showing that only 10% of the hydrogen consumed originate electrolysis. You need to create a corresponding parameter to represent this in the model.

3. **Specific Diesel Vehicle Data:** Another colleague noted that diesel vehicles can vary significantly in fuel consumption, and the LCA should account for differences between high and low-consuming ICEVs. Also, a notable blend share of ethanol should be represented (approximately 5%), which is not the case currently.

4. **Urban vs. Non-Urban Driving Conditions:** Team members highlighted that the human health impacts of gasoline vehicle emissions vary significantly between urban and non-urban settings. They are interested in understanding how these different driving environments influence the assessment results.

5. **Linking Emissions to Gasoline Consumption:** There was a particular interest in tracking specific emissions linked to gasoline consumption, especially $\text{CO}_{2}$, $\text{N}_{2}{O}$ and $\text{PM}_{2.5}$. The team wants to understand how assumptions about these emissions influence the overall impact assessment.

| Parameter                   | Unit                | Distribution | Loc    | Scale | Shape | Minimum | Maximum |
|-----------------------------|---------------------|--------------|--------|-------|-------|---------|---------|
| electricity_consumption     | kWh/km              | Normal       | 0.15   | 0.03  |       |         |         |
| battery_size                | kWh                 | Triangular   | 48     |       |       | 40      | 80      |
| energy_cell_density         | kWh/kg cell         | Triangular   | 0.25   |       |       | 0.2     | 0.3     |
| share_BoP                   | fraction            | Uniform      | 0.3    |       |       | 0.1     | 0.4     |
| share_BoP                   | fraction            | Uniform      | 0.3    |       |       | 0.1     | 0.4     |
| h2_tank_gravimetric_density | fraction            | Triangular   | 0.05   |       || 0.025   | 0.1     |
| h2_consumption              | kg H2/100 km        | Triangular   | 0.0115 |       || 0.008   | 0.015   |
| h2_electrolysis_share       | kg H2/kg H2         | Uniform      | 0.1    |       || 0.05    | 0.25    |
| gasoline_consumption        | L/100 km            | Normal       | 6.0    | 2.0   ||     |     |
| urban_driving               | fraction            | Uniform      |        |       |       | 0       | 1       |
| co2_gasoline                | kg_co2/kg_gasoline  | Weibull      | 3      | 3.16  | 2     |         |         |
| n2o_gasoline                | kg_n2o/kg_gasoline  | Lognormal    | -7     | 0.05  |       |         |         |
| pm25_gasoline               | kg_pm25/kg_gasoline | Triangular   | 0.0015 |       |       | 0.0005  | 0.003   |
| biofuel_fraction            | fraction            | Uniform      | 0.05   |       |       | 0.01    | 0.1     |


Documentation for uncertain parameters and classes for Monte Carlo sampling in BW2: https://stats-arrays.readthedocs.io/en/latest/

## 1. Set project

- This assigment requires a project with an existing ecoinvent database and corresponding biosphere. We can use ``training-day`` for this purpose.
- We want to import in `activity-browser` the passenger cars database contained in `lci-passenger-cars.xlsx`.

## 2. Import Excel database

We could also use direct the brightway module `bw2io` to import the database. It is sometimes easier to debug the import process using `bw2io` directly.

In [ ]:
imp = bi.ExcelImporter("lci-passenger-cars.xlsx")
imp.apply_strategies()
imp.match_database('ecoinvent-3.10-cutoff', fields=('name', 'unit', 'reference product', 'location'))
imp.statistics()

In [ ]:
imp.write_database()

## 3. Importing parameterized database

After building your new inventory on top of 'lci-passenger-cars.xlsx'

👓 The activity "driving the EV" gives a hint on how to add parameters and formulas

In [73]:
bd.projects.set_current("pisa-day")

In [68]:
bi.create_core_migrations()

In [75]:
imp = bi.ExcelImporter("solution/lci-passenger-cars_v2.xlsx")
imp.apply_strategies()
imp.match_database('ecoinvent-3.10.1-cutoff', fields=('name', 'unit', 'reference product', 'location'))
imp.match_database('lci-fuel-cell', fields=('name', 'unit', 'reference product', 'location'))
imp.match_database('lci-h2-tank', fields=('name', 'unit', 'reference product', 'location'))
imp.match_database('lci-hydrogen-electrolysis', fields=('name', 'unit', 'reference product', 'location'))
imp.statistics()

Extracted 1 worksheets in 0.02 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 2.50 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
6

(6, 110, 0)

---

#### ⚠️ **Unlinked exchanges?**

If so, let's run to get a list of unlinked exchanges in an Excel file:

In [49]:
#imp.write_excel()
#imp.write_excel(only_unlinked=True)

Wrote matching file to:
/Users/romain/Library/Application Support/Brightway3/pisa-day.38ea06373c2e801a3ec7d2103a700283/output/db-matching-lci-passenger-cars-v2.xlsx


This usually a great help to spot what went wrong.

---

#### 🚨 The syntax changes a bit when we want to work with parameters

In [76]:
if "lci-passenger-cars-v2" in bd.databases:
    del bd.databases["lci-passenger-cars-v2"]

imp.write_project_parameters()
imp.write_database(activate_parameters=True)

Title: Writing activities to SQLite3 database:
  Started: 04/21/2025 21:29:24
  Finished: 04/21/2025 21:29:24
  Total time elapsed: 00:00:00
  CPU %: 97.20
  Memory %: 2.74
Created database: lci-passenger-cars-v2


In [ ]:
bd.databases

### **Time to continue the analysis on AB!**

--------------------

----